<a href="https://colab.research.google.com/github/Murphy-Mary/MyProject/blob/main/game_recommendations_on_steam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import data


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

!kaggle datasets download -d antonkozyriev/game-recommendations-on-steam
!unzip /content/game-recommendations-on-steam.zip && rm /content/game-recommendations-on-steam.zip

Mounted at /content/gdrive
 93% 105M/112M [00:01<00:00, 93.0MB/s] 
100% 112M/112M [00:01<00:00, 65.8MB/s]
Archive:  /content/game-recommendations-on-steam.zip
  inflating: games.csv               
  inflating: games_metadata.json     
  inflating: recommendations.csv     
  inflating: users.csv               


In [2]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [71.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:14 http://ppa.launchpad.net/gr

In [3]:
#Get Install the Java Development Kit
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Get Spark lib -- address need to be change according to the current version of Spark: https://downloads.apache.org/spark/
!wget  http://apache.osuosl.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
#Unzip th Spark
!tar xf spark-3.3.2-bin-hadoop3.tgz
!rm spark-3.3.2-bin-hadoop3.tgz
#install findspark for Python
!pip install -q findspark
#Install extra lib(s)
!pip install -q xlrd
!pip install -q kaggle
!pip install -q kora

#import and set environment for spark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"
import kora

import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession


SPARK_URL = "local[*]"

spark = SparkSession.builder.master(SPARK_URL).getOrCreate()

# Check the pyspark version
import pyspark
print(pyspark.__version__)

--2023-02-22 06:59:51--  http://apache.osuosl.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
Resolving apache.osuosl.org (apache.osuosl.org)... 140.211.166.134, 64.50.236.52, 64.50.233.100, ...
Connecting to apache.osuosl.org (apache.osuosl.org)|140.211.166.134|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299360284 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.2-bin-hadoop3.tgz’

spark-3.3.2-bin-had 100%[===================>] 285.49M  34.3MB/s    in 39s     

2023-02-22 07:00:30 (7.31 MB/s) - ‘spark-3.3.2-bin-hadoop3.tgz’ saved [299360284/299360284]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.7 MB/s eta 0:00:00
3.3.2


In [4]:
!pip install pyspark_dist_explore

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import numpy as np 
import pandas as pd
import pyspark.sql.functions as f
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display, HTML, display_html #usefull to display wide tables
from pyspark_dist_explore import Histogram, hist, distplot, pandas_histogram
from pyspark.sql import Row

from pyspark.sql.functions import lit
from pyspark import SparkConf
from pyspark import SparkContext
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

#sc = pyspark.SparkContext()
sqlContext = pyspark.SQLContext(sc)
%matplotlib inline

/content/spark-3.3.2-bin-hadoop3/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [6]:
game_tags = spark.read.json("/content/games_metadata.json")
game_tags.printSchema()
game_tags.show()

root
 |-- app_id: long (nullable = true)
 |-- description: string (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)

+------+--------------------+--------------------+
|app_id|         description|                tags|
+------+--------------------+--------------------+
| 10090|Call of Duty is b...|[Zombies, World W...|
| 13500|Enter the dark un...|[Action, Adventur...|
| 22364|                    |            [Action]|
|113020|Monaco: What's Yo...|[Co-op, Stealth, ...|
|226560|Escape Dead Islan...|[Zombies, Adventu...|
|249050|Dungeon of the En...|[Roguelike, Strat...|
|250180|“METAL SLUG 3”, t...|[Arcade, Classic,...|
|253980|Experience incred...|[RPG, Action, Fan...|
|261550|A strategy/action...|[Medieval, Strate...|
|271850|                    |[Strategy, Simula...|
|282900|Packed with fast-...|[Anime, JRPG, Fem...|
| 19810|Lead a team of do...|  [Action, Tactical]|
| 15270|In a ferocious Ar...|[Horror, Action, ...|
| 21130|Build th

In [7]:
game_tags.count()

21236

In [8]:
game = spark.read.csv('/content/games.csv',header=True)
game.printSchema()
game.show()

root
 |-- app_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- date_release: string (nullable = true)
 |-- win: string (nullable = true)
 |-- mac: string (nullable = true)
 |-- linux: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- positive_ratio: string (nullable = true)
 |-- user_reviews: string (nullable = true)
 |-- price_final: string (nullable = true)
 |-- price_original: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- steam_deck: string (nullable = true)

+------+--------------------+------------+----+-----+-----+---------------+--------------+------------+-----------+--------------+--------+----------+
|app_id|               title|date_release| win|  mac|linux|         rating|positive_ratio|user_reviews|price_final|price_original|discount|steam_deck|
+------+--------------------+------------+----+-----+-----+---------------+--------------+------------+-----------+--------------+--------+----------+
| 10090|Call of 

In [9]:
game.count()

21236

In [10]:
recomm = spark.read.csv('/content/recommendations.csv',header=True)
recomm.printSchema()
recomm.show()

root
 |-- app_id: string (nullable = true)
 |-- helpful: string (nullable = true)
 |-- funny: string (nullable = true)
 |-- date: string (nullable = true)
 |-- is_recommended: string (nullable = true)
 |-- hours: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- review_id: string (nullable = true)

+-------+-------+-----+----------+--------------+-----+-------+---------+
| app_id|helpful|funny|      date|is_recommended|hours|user_id|review_id|
+-------+-------+-----+----------+--------------+-----+-------+---------+
| 975370|      0|    0|2022-12-12|          true| 36.3|  14546|        0|
| 304390|      4|    0|2017-02-17|         false| 11.5|    797|        1|
|1085660|      2|    0|2019-11-17|          true|336.5|  66915|        2|
| 703080|      0|    0|2022-09-23|          true| 27.4|  68251|        3|
| 526870|      0|    0|2021-01-10|          true|  7.9|   6647|        4|
| 306130|      0|    0|2021-10-10|          true|  8.6|  12866|        5|
| 238960|      

In [11]:
recomm.count()

6126803

In [12]:
user = spark.read.csv('/content/users.csv',header=True)
user.printSchema()
user.show()

root
 |-- user_id: string (nullable = true)
 |-- products: string (nullable = true)
 |-- reviews: string (nullable = true)

+-------+--------+-------+
|user_id|products|reviews|
+-------+--------+-------+
|4154727|     156|      1|
|2637640|     329|      2|
|1438551|     176|      1|
|1549343|      98|      2|
|1712796|     144|      2|
|2832252|     229|      1|
|3004113|     187|      1|
|3128965|    1824|      1|
|3274807|      12|      1|
|3395007|    1319|      1|
|3407667|    1903|      3|
|3619475|     352|      1|
|3788372|     107|      1|
|4067577|     587|      3|
| 274637|     321|      2|
| 360193|     166|      1|
| 546655|     752|      1|
| 853991|     179|      1|
|1119958|     325|      1|
|1266621|     115|      1|
+-------+--------+-------+
only showing top 20 rows



In [13]:
user.count()

4240753

# Prepare Section

In [14]:
game_tng = game.join(game_tags,["app_id"],"full")

In [15]:
game_tng.show()
# game_tng.select('tags').collect()[0][0][0]
# game_tng.describe().show()

+------+--------------------+------------+----+-----+-----+--------------------+--------------+------------+-----------+--------------+--------+----------+--------------------+--------------------+
|app_id|               title|date_release| win|  mac|linux|              rating|positive_ratio|user_reviews|price_final|price_original|discount|steam_deck|         description|                tags|
+------+--------------------+------------+----+-----+-----+--------------------+--------------+------------+-----------+--------------+--------+----------+--------------------+--------------------+
|    10|      Counter-Strike|  2000-11-01|true| true| true|Overwhelmingly Po...|            97|      132250|       9.99|          9.99|     0.0|      true|Play the world's ...|[Action, FPS, Mul...|
|    50|Half-Life: Opposi...|  1999-11-01|true| true| true|Overwhelmingly Po...|            95|       14057|       4.99|          4.99|     0.0|      true|Return to the Bla...|[FPS, Action, Cla...|
|   130|Ha

In [16]:
game_tng = game_tng.filter(game_tng.app_id.isNotNull()\
                           |game_tng.title.isNotNull()\
                           |game_tng.win.isNotNull()\
                           |game_tng.rating.isNotNull()\
                           |game_tng.price_final.isNotNull()\
                           |game_tng.price_original.isNotNull()\
                           |game_tng.description.isNotNull()\
                           |game_tng.tags.isNotNull()\
                           ).sort("app_id") 


In [17]:
game_tng.count()

21236

In [18]:
game_tng.show()

+-------+--------------------------+------------+----+-----+-----+--------------------+--------------+------------+-----------+--------------+--------+----------+------------------------------+--------------------+
| app_id|                     title|date_release| win|  mac|linux|              rating|positive_ratio|user_reviews|price_final|price_original|discount|steam_deck|                   description|                tags|
+-------+--------------------------+------------+----+-----+-----+--------------------+--------------+------------+-----------+--------------+--------+----------+------------------------------+--------------------+
|     10|            Counter-Strike|  2000-11-01|true| true| true|Overwhelmingly Po...|            97|      132250|       9.99|          9.99|     0.0|      true|          Play the world's ...|[Action, FPS, Mul...|
|1000000|                 ASCENXION|  2021-05-14|true|false|false|     Mostly Positive|            72|          22|       2.49|          9.9

-----------------------------------------------------


In [19]:
user_recc = recomm.join(user,["user_id"],"full")

In [20]:
user_recc.show()
# user_recc.describe().show()

+-------+-------+-------+-----+----------+--------------+-----+---------+--------+-------+
|user_id| app_id|helpful|funny|      date|is_recommended|hours|review_id|products|reviews|
+-------+-------+-------+-----+----------+--------------+-----+---------+--------+-------+
|1000015|1086940|      0|    0|2022-04-28|          true|148.5|  5934747|      44|      1|
|1000030| 438100|      0|    0|2022-02-17|          true|  9.4|  5702748|       3|      1|
|1000032|1817070|      0|    0|2022-09-11|          true| 61.9|  4292020|      11|      1|
|1000054| 275850|      0|    0|2016-10-14|         false|136.0|  3811170|     263|      1|
|1000061| 289070|      0|    0|2020-08-01|         false|281.3|  2075815|      42|      1|
|1000064|  39210|      3|    0|2022-01-03|          true|899.4|  2740685|     133|      1|
|1000067|1086940|      0|    0|2020-10-06|          true| 96.5|   765191|      78|      1|
|1000070|1144200|      0|    0|2021-12-28|          true| 70.2|  6053316|     154|      1|

In [21]:
user_recc = user_recc.filter(user_recc.app_id.isNotNull()).sort("user_id") 

In [22]:
user_recc.count()

6126803

In [23]:
user_recc.show()

+-------+-------+-------+-----+----------+--------------+-----+---------+--------+-------+
|user_id| app_id|helpful|funny|      date|is_recommended|hours|review_id|products|reviews|
+-------+-------+-------+-----+----------+--------------+-----+---------+--------+-------+
|      0| 291550|      0|    0|2020-11-01|          true| 12.7|   104565|      54|      2|
|      0| 493520|      0|    0|2020-10-30|          true| 18.6|  4723376|      54|      2|
|      1| 394360|      0|    0|2021-11-26|          true|107.0|   306001|     369|      1|
|    100|1172470|      0|    0|2021-07-10|         false|404.0|   434065|       0|      1|
|   1000| 431960|      0|    0|2020-04-05|          true| 80.9|   276676|     182|      8|
|   1000| 440900|      0|    0|2018-07-21|          true|136.3|   124995|     182|      8|
|   1000| 386360|      0|    0|2019-10-14|          true| 81.0|  4276553|     182|      8|
|   1000| 438100|      0|    0|2018-07-16|          true| 29.5|  3307295|     182|      8|

In [24]:
all_data = game_tng.join(user_recc,["app_id"],"full")

In [25]:
all_data.printSchema()

root
 |-- app_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- date_release: string (nullable = true)
 |-- win: string (nullable = true)
 |-- mac: string (nullable = true)
 |-- linux: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- positive_ratio: string (nullable = true)
 |-- user_reviews: string (nullable = true)
 |-- price_final: string (nullable = true)
 |-- price_original: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- steam_deck: string (nullable = true)
 |-- description: string (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- user_id: string (nullable = true)
 |-- helpful: string (nullable = true)
 |-- funny: string (nullable = true)
 |-- date: string (nullable = true)
 |-- is_recommended: string (nullable = true)
 |-- hours: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- products: string (nullable = true)
 |-- reviews: string (nullabl

In [26]:
# all_data.count()
all_data.show()
# app_id,title,win

+-------+--------------------+------------+----+-----+-----+--------------------+--------------+------------+-----------+--------------+--------+----------+--------------------+--------------------+-------+-------+-----+----+--------------+-----+---------+--------+-------+
| app_id|               title|date_release| win|  mac|linux|              rating|positive_ratio|user_reviews|price_final|price_original|discount|steam_deck|         description|                tags|user_id|helpful|funny|date|is_recommended|hours|review_id|products|reviews|
+-------+--------------------+------------+----+-----+-----+--------------------+--------------+------------+-----------+--------------+--------+----------+--------------------+--------------------+-------+-------+-----+----+--------------+-----+---------+--------+-------+
|1000030|Cook Serve Delici...|  2020-10-14|true| true|false|Overwhelmingly Po...|            95|        1396|      19.99|         19.99|     0.0|      true|Cook, serve and m...|[

In [27]:
#Drop Null
all_data = all_data.filter(all_data.app_id.isNotNull()|all_data.title.isNotNull()|all_data.win.isNotNull()).sort("app_id") 

In [28]:
# all_data.count()

6147826

In [29]:
all_data.show()

+-------+--------------------------+------------+----+-----+-----+--------------------+--------------+------------+-----------+--------------+--------+----------+------------------------------+--------------------+-------+-------+-----+----+--------------+-----+---------+--------+-------+
| app_id|                     title|date_release| win|  mac|linux|              rating|positive_ratio|user_reviews|price_final|price_original|discount|steam_deck|                   description|                tags|user_id|helpful|funny|date|is_recommended|hours|review_id|products|reviews|
+-------+--------------------------+------------+----+-----+-----+--------------------+--------------+------------+-----------+--------------+--------+----------+------------------------------+--------------------+-------+-------+-----+----+--------------+-----+---------+--------+-------+
|     10|            Counter-Strike|  2000-11-01|true| true| true|Overwhelmingly Po...|            97|      132250|       9.99|   

In [30]:
all_data.filter(all_data.date_release.isNull()).show()

+------+-----+------------+---+---+-----+------+--------------+------------+-----------+--------------+--------+----------+-----------+----+-------+-------+-----+----+--------------+-----+---------+--------+-------+
|app_id|title|date_release|win|mac|linux|rating|positive_ratio|user_reviews|price_final|price_original|discount|steam_deck|description|tags|user_id|helpful|funny|date|is_recommended|hours|review_id|products|reviews|
+------+-----+------------+---+---+-----+------+--------------+------------+-----------+--------------+--------+----------+-----------+----+-------+-------+-----+----+--------------+-----+---------+--------+-------+
+------+-----+------------+---+---+-----+------+--------------+------------+-----------+--------------+--------+----------+-----------+----+-------+-------+-----+----+--------------+-----+---------+--------+-------+



In [31]:
all_data.groupBy("app_id").count().filter("count > 1").sort("app_id").show()

+-------+------+
| app_id| count|
+-------+------+
|1044720|  4265|
| 105600| 48595|
|1056640|   264|
|1063660|  3858|
|1063730|  4482|
|1065310|  1763|
| 107410|115929|
|1085660| 33136|
| 108600| 45965|
|1086940| 37878|
|1091500|109328|
|1092790|  9545|
|1096530| 10200|
|1113120|  3461|
|1142710| 32226|
|1144200| 50146|
|1145360| 11859|
|1151340| 42217|
|1151640| 12444|
|1158310| 10610|
+-------+------+
only showing top 20 rows



In [32]:
# all_data.write.parquet("/content/gdrive/MyDrive/sizeproject/allgamerecc.parquet")

# EDA Section

In [34]:
all_data.show()

+------+--------------------+------------+----+----+-----+--------------------+--------------+------------+-----------+--------------+--------+----------+--------------------+--------------------+-------+-------+-----+----------+--------------+-----+---------+--------+-------+
|app_id|               title|date_release| win| mac|linux|              rating|positive_ratio|user_reviews|price_final|price_original|discount|steam_deck|         description|                tags|user_id|helpful|funny|      date|is_recommended|hours|review_id|products|reviews|
+------+--------------------+------------+----+----+-----+--------------------+--------------+------------+-----------+--------------+--------+----------+--------------------+--------------------+-------+-------+-----+----------+--------------+-----+---------+--------+-------+
|238460|BattleBlock Theater®|  2014-05-15|true|true| true|Overwhelmingly Po...|            97|       50984|      14.99|         14.99|     0.0|      true|Welcome to B

In [36]:
all_data.groupBy("app_id","title").count().sort("count").show()

+-------+--------------------+-----+
| app_id|               title|count|
+-------+--------------------+-----+
| 325150|          Run or Die|    1|
| 750170|Diesel Railcar Si...|    1|
| 352220| King of Dragon Pass|    1|
|1119840|       Sands of Aura|    1|
| 361850|Portal of Evil: S...|    1|
|1173200|          Metal Unit|    1|
|  38170|      Cradle of Rome|    1|
|1265620|Car Mechanic Simu...|    1|
| 397460|The Jackbox Party...|    1|
|1416670|PC Building Simul...|    1|
| 426930|            Moonshot|    1|
|1547410|DnD Adventure: Wi...|    1|
| 447120|Where the Water T...|    1|
|1585790|CrossCode Manlea ...|    1|
| 461640|Sins Of The Demon...|    1|
|1768280|Ozymandias: Bronz...|    1|
| 483420|          Adam Wolfe|    1|
|1822450|The Slaverian Tru...|    1|
| 488060|Age of Empires II...|    1|
| 208807|Tomb Raider: Avia...|    1|
+-------+--------------------+-----+
only showing top 20 rows

